# 0. Зоны видимости

у питона есть 4 зоны видимостиЖ

- built-in - зона видимости, которая создается, когда я впервые запускаю скрипт 
- global 
- enclosing
- local

In [4]:
dir(__builtins__)[-5:]

['super', 'tuple', 'type', 'vars', 'zip']

In [6]:
x = 5

In [9]:
globals()['_i6']

'x = 5'

In [10]:
a = 5
print(a)

def f(x):
    a = 10    # всё что произошло внутри функции остается внутри функции
    print(a)
    return a**2

f(5)
print(a)

5
10
5


In [11]:
a = 5
print(a)

# !!! никогда так не делайте !!! 

def f(x):
    global a # я обращаюсь к глобальной переменной
    a = 10    
    print(a)
    return a**2

f(5)
print(a)

5
10
10


In [12]:
def f():
    a = 1
    def g():
        a = 2
        print(a)
    g()
    print(a)
    
f()

2
1


In [13]:
def f():
    print(it)
    
def q(func):
    for it in range(10):
        func()
q(f)

NameError: name 'it' is not defined

In [ ]:
# python - в нем новую зону видимости создает только функция
# c++ циклы и условия тоже создают свои зоны видимости

# 1. Декораторы

In [14]:
# замыкание (фабрика функций)

def make_adder(y):
    def adder(x):
        return x + y
    return adder

add_two = make_adder(2)
add_five = make_adder(5)

In [15]:
add_two(7)

9

In [16]:
add_five(7)

12

In [41]:
import sys

def deprecate(func):
    def wrapper(*args, **kwargs):
        print(f'WARNING! {func.__name__} is deprecated', file=sys.stderr)
        
        return func(*args, **kwargs)
    return wrapper

In [42]:
print(5)

5


In [43]:
pprint = deprecate(print)
pprint(5)

5


WARNING! print is deprecated


In [30]:
@deprecate
def f(a, b):
    return a + b

In [31]:
f(4, 5)

WARNING! f is deprecated


9

In [34]:
from IPython import display

def bananize(func):
    return display.HTML('<img src="http://www.sherv.net/cm/emo/funny/2/big-dancing-banana-smiley-emoticon.gif">')

@bananize
def show(x):
    print(x)

In [36]:
show

In [37]:
# Не корректный синтаксис! Это не фабрика функций!
show(4)

TypeError: 'HTML' object is not callable

У такого подхода есть проблема, затираются докстринги (((

In [44]:
def deprecate(func):
    def wrapper(*args, **kwargs):
        print(f'WARNING! {func.__name__} is deprecated', file=sys.stderr)
        
        return func(*args, **kwargs)
    return wrapper

In [47]:
@deprecate
def show(x):
    '''
    Тут написана документация
    '''
    print(x)

In [48]:
show()

WARNING! show is deprecated


TypeError: show() missing 1 required positional argument: 'x'

__Решение 1:__

In [49]:
def deprecate(func):
    def wrapper(*args, **kwargs):
        print(f'WARNING! {func.__name__} is deprecated', file=sys.stderr)
        return func(*args, **kwargs)
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    wrapper.__module__ = func.__module__
    return wrapper

In [50]:
@deprecate
def show(x):
    '''
    Тут написана документация
    '''
    print(x)

In [ ]:
show()

__Решение 2:__

In [51]:
import functools # модуль с кучей декораторов

In [52]:
def deprecate(func):
    @functools.wraps(func) # перекопирует доку и тп
    def wrapper(*args, **kwargs):
        print(f'WARNING! {func.__name__} is deprecated', file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper

In [53]:
@deprecate
def show(x):
    '''
    Тут написана документация
    '''
    print(x)

In [ ]:
show()

__Упражнение:__ написать декоратор, который будет слендить за тем, что функция вызывается только один раз

In [63]:
class NotOnceError(Exception):
    def __init__(self, message):
        super().__init__(message)

def once(func):
    called = False
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        nonlocal called 
        if not called:
            called = True
            return func(*args, **kwargs)
        else:
            raise NotOnceError('Function has allready called')
    return wrapper

In [64]:
@once
def f(a, b):
    return a + b

In [65]:
f(4, 5)

9

In [66]:
f(5, 5)

NotOnceError: Function has allready called

Посмотрите на досуге, что есть в модуле [functools](https://pythonworld.ru/moduli/modul-functools.html)

# 2. Декораторы внутри классов

[Семинар с ИАД про классы](https://github.com/hse-ds/iad-intro-ds/blob/master/2021/seminars/sem04_oop.ipynb)

In [67]:
class Thermometer:
    """
    Thermometer containing temperature in Celsius 
    :param temperature: temperature to contain
    """
    
    def __init__(self, temperature: float) -> None:
        self.temperature = temperature

In [71]:
TTT = Thermometer(-10000)
TTT.temperature

-10000

In [72]:
class Thermometer:
    """
    Thermometer containing temperature in Celsius 
    :param temperature: temperature to contain
    """
    MINIMAL_TEMPERATURE = -273.15
    def __init__(self, temperature: float) -> None:
        self.set_temperature(temperature)
        
    def set_temperature(self, value: float) -> None:
        if value < Thermometer.MINIMAL_TEMPERATURE: 
            raise ValueError(f"Temperature cannot be less than {Thermometer.MINIMAL_TEMPERATURE}")
        self.temperature = value
        
thermometer = Thermometer(10.)
thermometer.set_temperature(-100000.)

ValueError: Temperature cannot be less than -273.15

In [73]:
class Thermometer:
    """
    Thermometer containing temperature in Celsius 
    :param temperature: temperature to contain
    """
    MINIMAL_TEMPERATURE = -273.15
    def __init__(self, temperature: float) -> None:
        self.temperature = temperature
    
    @property
    def temperature(self) -> float:
        return self._temperature
    
    @temperature.setter
    def temperature(self, value: float) -> None:
        if value < Thermometer.MINIMAL_TEMPERATURE: 
            raise ValueError(f"Temperature cannot be less than {Thermometer.MINIMAL_TEMPERATURE}")
        self._temperature = value
        
thermometer = Thermometer(10.)
thermometer.temperature = -100000.

ValueError: Temperature cannot be less than -273.15

In [84]:
class Thermometer:
    """
    Thermometer containing temperature in Celsius 
    :param temperature: temperature to contain
    """
    MINIMAL_TEMPERATURE = -273.15
    def __init__(self, temperature: float) -> None:
        self.temperature = temperature
    
    @property
    def temperature(self) -> float:
        return self._temperature
    
    @temperature.setter
    def temperature(self, value: float) -> None:
        if value < Thermometer.MINIMAL_TEMPERATURE: 
            raise ValueError(f"Temperature cannot be less than {Thermometer.MINIMAL_TEMPERATURE}")
        self._temperature = value
    
    @staticmethod
    def celsius_to_fahrenheit(value) -> float:
        return value * 1.8 + 32
        
    def get_fahrenheit(self) -> float:
        return self.celsius_to_fahrenheit(self.temperature)

In [85]:
Thermometer.celsius_to_fahrenheit(50)

122.0

# 3. Генераторы

Словарик: 

* __Итератор__ - объект перечислитель, который выводит каждый элемент по очереди
* __Генератор__ - подвид итераторов, который перебирает элементы, но не индексирует их (грубо говоря это просто функция с `yield` вместо `return`

Примеры встроенных генераторов: `enumerate` и `range`

Итерирование - просто тупо перебор. 

In [86]:
num_list = [11, 22, 33]

for j,item in enumerate(num_list):
    print(j, item)

0 11
1 22
2 33


Можно переписать это через методы `iter` и `next`:

In [88]:
itr = iter(num_list)

In [89]:
next(itr)

11

In [90]:
next(itr)

22

In [91]:
next(itr)

33

In [92]:
next(itr)

StopIteration: 

Когда кончились объекты, выскакивает исключение `StopIteration`. Цикл обрабатывает это исключение незаметно для нас. 

__Задача:__ реализовать функцию fuzzysearch

Проверить, является ли первая строка подпоследовательностью второй

Пример работы:

```
fuzzysearch('car', 'cartwheel');        // true
fuzzysearch('cwhl', 'cartwheel');       // true
fuzzysearch('cwhee', 'cartwheel');     // true
fuzzysearch('cartwheel', 'cartwheel');  // true
fuzzysearch('cwheeel', 'cartwheel');    // false
fuzzysearch('lw', 'cartwheel');         // false
```

In [93]:
def fuzzysearch(word: str, text: str) -> bool:
    i = 0
    for j in range(len(text)):
        if word[i] == text[j]:
            i += 1
            if i == len(word):
                return True
    return False

In [94]:
def fuzzysearch(word: str, text: str) -> bool:
    sub_string = iter(word)
    main_string = iter(text)

    cur_sub =  next(sub_string, None)
    cur_main = next(main_string, None)

    while cur_main and cur_sub:
        if cur_sub == cur_main:
            cur_sub =  next(sub_string, None)
            cur_main = next(main_string, None)
        else:
            cur_main = next(main_string, None)

    if cur_sub:
        return False
    else:
        return True

In [95]:
assert fuzzysearch('car', 'cartwheel')
assert fuzzysearch('cwhl', 'cartwheel')
assert fuzzysearch('cwhee', 'cartwheel')
assert fuzzysearch('cartwheel', 'cartwheel')
assert not fuzzysearch('cwheeel', 'cartwheel')
assert not fuzzysearch('lw', 'cartwheel')

Можно придумать свой итератор. Для этого надо написать класс с двумя методами `__iter__()` и  `__next__()`.

* __iter__ возвращает объект для итерирования 

* __next__ озвращает новые элементы по ходу итерирования

In [98]:
class my_range:
    
    def __init__(self, low, high):
        self.current = low - 1
        self.high = high + 1

    def __iter__(self):
        return self

    def __next__(self):
        self.current += 1
        if self.current < self.high:
            return self.current
        else:
            raise StopIteration

In [99]:
for c in my_range(100, 105):
    print(c)

100
101
102
103
104
105


In [100]:
for c in range(100, 105):
    print(c)

100
101
102
103
104


Напишем генератор для чисел Фиббоначи

$$
a_1 = 0, a_2 = 1, a_3 = a_1 + a_2, a_4 = a_3 + a_2
$$

In [102]:
class FibonacciGenerator():
    def __init__(self, n):
        self.n = n
        self.cnt = 0
        self.prev = 0
        self.cur = 1
        
    def __iter__(self):
        return self

    def __next__(self):
        if self.cnt < self.n:
            self.cnt += 1
            result = self.prev
            self.prev, self.cur = self.cur, self.prev + self.cur
            return result
        else:
            raise StopIteration
        
gen = FibonacciGenerator(10)

for i in gen:
    print(i)

0
1
1
2
3
5
8
13
21
34


Используя `yield` можно сильно упростить реализацию и переписать генератор в виде функции. 

In [103]:
def fibonacci():
    prev, cur = 0, 1
    while True:
        yield prev
        prev, cur = cur, prev + cur

for i in fibonacci():
    print(i)
    if i > 10:
        break

0
1
1
2
3
5
8
13


In [104]:
it = iter(fibonacci())

In [116]:
next(it)

89

Про то как работает `yield`:

In [118]:
def gen_fun():
    
    print('block 1')
    yield 1
    
    print('block 2')
    yield 2
    
    print('end')

In [120]:
a = iter(gen_fun())

In [124]:
next(a)

end


StopIteration: 

__Происходит следующее:__

1. при вызове функции __gen_fun__ создается объект-генератор
2. __for__ вызывает __iter()__ с этим объектом и получает итератор этого генератора
3. в цикле вызывает функция __next()__ с этим итератором пока не будет получено исключение __StopIteration__
4. при каждом вызове __next__ выполнение в функции начинается с того места где было завершено в последний раз и продолжается до следующего __yield__

In [125]:
def gen_fun_1():
    print('block 1')
    return 1

def gen_fun_2():
    print('block 2')
    return 2

def gen_fun_3():
    print('end')

def gen_fun_end():
    raise StopIteration

Когда интерпретатор доходит до ключевого слова `return`, выполнение функции полностью прекращается. Но когда он доходит до ключевого слова `yield`, программа приостанавливает выполнение функции и возвращает значение в итерируемый объект. После этого интерпретатор возвращается к генератору, чтобы повторить процесс для нового значения.

Можно написать свой собственный `range`:

In [126]:
def cool_range(start, stop, inc):
    x = start
    while x < stop:
        yield x
        x += inc

for n in cool_range(1, 5, 0.5):
    print(n)

1
1.5
2.0
2.5
3.0
3.5
4.0
4.5
